In [1]:
import numpy as np
import sys
import os
sys.path.append(os.path.join(os.getcwd(),"../Algorithm encapsulation"))
sys.path.append(os.path.join(os.getcwd(),".."))
print(sys.path)
from LMPband import LMPband
import pandas as pd
from ga_platoon import max_dt

['f:\\workspace\\jupyter\\论文\\graduation thesis\\Case Analysis', 'd:\\anaconda\\envs\\cplex\\python37.zip', 'd:\\anaconda\\envs\\cplex\\DLLs', 'd:\\anaconda\\envs\\cplex\\lib', 'd:\\anaconda\\envs\\cplex', '', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\chardet-3.0.4-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\docloud-1.0.375-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\urllib3-1.25.11-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\fonttools-4.36.0-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\cvxopt-1.3.0-py3.7-win-amd64.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\pytz-2022.2.1-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\win32', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\win32\\lib', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\Pythonwin', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\0\\.ipython', 'f:\\wor

In [2]:
def get_rf(d,p):
    tmp=[]
    for i,a in enumerate(p):
        idx=np.where(a!=0)[0]
        a=a&0
        if len(idx)>0:
            a[:idx[0]]=1
        tmp.append((d[i]*a).sum())
    return tmp

In [3]:
phase = np.loadtxt("./data/phase")
cir=phase[0].sum()
phase=phase/cir
vol = np.loadtxt("./data/volume")
qv = np.loadtxt("./data/path_vol")
phase_get = np.load("./data/phase_get.npy")
d = np.loadtxt("./data/distance")
straight_get = np.load("./data/straight_get.npy")
ison = np.array([0, 0, 0, 1, 1, 1])
px = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
pxb = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
qb=[120,120]
lower, upper = 8, 14
linspace = np.arange(lower, upper, 1)
lin_num = len(linspace)

# 南北直行方向绿灯时间
sg=np.array([(straight_get[i]*phase).sum(axis=1) for i in range(len(straight_get))])
rho = vol[0] / vol[1]
num = len(vol[0])
numr = len(qv)
nump = len(phase_get)
M = 1e6
nx = 0.00000001
be = 8
speedScope = np.array([lower, upper])
speedVar = np.array([-2, 2])

g=np.array([(phase_get[i]*phase).sum(axis=1) for i in range(numr) ])
r=1-g
rf=np.array([get_rf(phase,phase_get[i]) for i in range(numr)])

ex=[1,0,1,0,1,0,1,1,1,1,1,0]
dwt=15
qb_x=np.ones_like(sg)*20
cap=np.ones_like(sg,dtype=int)*3

spd_on=pd.read_csv(os.path.join(os.getcwd(),"..","speed analysis","上行速度数据.csv"),index_col=0)
spd_in=pd.read_csv(os.path.join(os.getcwd(),"..","speed analysis","下行速度数据.csv"),index_col=0)

In [4]:
lmpb=LMPband(phase, [80,180], vol, qv, phase_get, d, straight_get, ison, px, pxb, qb,qb_x,cap,ex, dwt, 
           linspace, be, speedScope,speedVar,speedScope,speedVar,spd_on,spd_in)

lmpb._M1_solve()


conflict(s): 0
None
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
1 of 1 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective -7487.0001.
Tried aggregator 3 times.
MIP Presolve eliminated 9294 rows and 286 columns.
MIP Presolve modified 1523 coefficients.
Aggregator did 957 substitutions.
Reduced MIP has 1116 rows, 597 columns, and 3982 nonzeros.
Reduced MIP has 152 binaries, 76 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (22.49 ticks)
Probing fixed 0 vars, tightened 51 bounds.
Probing time = 0.02 sec. (0.40 ticks)
Tried aggregator 2 times.
Detecting symmetries...
MIP Presolve eliminated 6 rows and 2 columns.
MIP Presolve modified 290 coefficients.
Aggregator did 3 substitutions.
Reduced MIP has 1107 rows, 592 columns, and 3953 nonzeros.
Reduced MIP has 148 binaries, 76 generals, 0 SOSs, and 0 indicators.
Presolve tim

In [5]:
lmpb._M2_solve()

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125

Multi-objective solve log . . .

Index  Priority  Blend          Objective      Nodes  Time (sec.)  DetTime (ticks)

Performing restart 1
    1         2      1   2.3588201786e+04       5869         1.31           848.73

Performing restart 1

Performing restart 2
    2         1      1   9.8559855593e+01      54370        12.56          9363.27
status  = multi-objective optimal
time    = 13.906 s.
problem = MILP
gap     = 4.23941e+72%

object value 23588


In [6]:
lmpb.get_dataframe()

,cross_number,distance,b1,b2,b3,b4,b5,b6,offset,p,...,u3,u4,u5,u6,bb1,bb2,dw1,dw2,tb1,tb2
0,1.0,800.0,26.137068,26.137068,0.000000,25.612711,0.000000,35.220163,0.000000,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,26.137068,35.220163,15.000000,15.000000,72.142857,94.689492
1,2.0,520.0,24.098006,24.098006,0.000000,34.107947,0.000000,34.107947,5.595287,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,24.098006,34.107947,15.000000,15.000000,64.045085,55.147360
2,3.0,500.0,24.098006,24.098006,0.000000,21.317467,0.000000,21.317467,5.595287,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,24.098006,21.317467,15.000000,15.000000,50.714286,62.500000
3,4.0,490.0,25.673645,25.673645,0.000000,21.317467,0.000000,21.317467,50.980206,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,25.673645,21.317467,15.000000,15.000000,35.000000,35.000000
4,5.0,370.0,20.400000,20.400000,52.000000,20.400000,0.000000,20.400000,73.413123,1.0,...,1.000000e-08,1.000000e-08,1.000000e-08,1.099998e-02,20.400000,20.400000,19.821429,15.000000,46.250000,49.550000
5,6.0,254.0,52.000000,52.000000,52.000000,20.400000,0.000000,20.400000,35.063124,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,52.000000,20.400000,15.000000,15.000000,18.142857,18.142857
6,7.0,585.0,24.259690,0.000000,24.259690,26.933893,0.000000,35.727907,62.878772,1.0,...,1.000000e-08,9.999999e-09,1.000000e-08,1.000000e-08,24.259690,35.727907,15.000000,30.000000,61.656783,79.637532
7,8.0,1020.0,24.259690,0.000000,24.259690,8.000000,0.000000,35.727907,136.003772,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,24.259690,24.259690,15.000000,15.000000,87.857143,87.857143
8,9.0,409.0,30.532043,0.000000,30.532043,24.957256,30.532043,0.000000,41.456668,1.0,...,1.000000e-08,1.000000e-08,2.720562e-02,1.000000e-08,30.532043,30.532043,15.000000,22.117235,51.125000,51.331520
9,10.0,547.0,38.831021,0.000000,38.831021,24.957256,30.532043,0.000000,59.334527,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,38.831021,30.532043,15.000000,15.000000,54.071429,54.071429


In [ ]:
from  SimAnalysis import SimAnalysis
df=lmpb.get_dataframe()
sim=SimAnalysis(df,phase,ex,os.path.join(os.getcwd(),"..","vissim","aa","zhongsan.inp"),
               os.path.join(os.getcwd(),"..","vissim","aa","vissim.ini"),
               os.path.join(os.getcwd(),"..","vissim","aa","zhongsan - 副本.inp"))
sim.start()
spdx=sim.analysis_speed(os.path.join(os.getcwd(),"..","vissim","aa","zhongsan - 副本.fzp"))
delayx=sim.analysis_delay(os.path.join(os.getcwd(),"..","vissim","aa",'zhongsan - 副本.vlz'))
tvltx=sim.analysis_traveltime(os.path.join(os.getcwd(),"..","vissim","aa",'zhongsan - 副本.rsz'))